<a href="https://colab.research.google.com/github/rahiakela/kaggle-competition-projects/blob/master/google-ai4code/01_exploratory_data_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Google AI4Code: Exploratory Data Analysis



**Exploratory data analysis is the work of a detective. Understanding the possibilities of your data is the first step in laying the groundwork for future modeling. With this notebook, we try to make sense of our data and demonstrate how data can be analyzed. We'll look for trends, limitations, and other characteristics linked to the questions we're interested in as part of our investigation.**

Reference:

https://www.kaggle.com/code/andreaspalmgren/ai4code-comprehensive-eda

##Setup

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
from tqdm import tqdm
from pathlib import Path
import re
import os

pd.options.display.width = 180
pd.options.display.max_colwidth = 100

data_dir = Path("../input/AI4Code")

rc = {"axes.spines.left" : True,
      "axes.spines.right" : False,
      "axes.spines.bottom" : True,
      "axes.spines.top" : False,
      "xtick.bottom" : True,
      "xtick.labelbottom" : True,
      "ytick.labelleft" : True,
      "ytick.left" : True,
      "figure.subplot.hspace" : 0.7,
    "figure.titleweight" : "bold",
    "axes.titleweight" : "bold",
     "font.weight" : "bold"}
plt.rcParams.update(rc)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
# content/gdrive/My Drive/Kaggle is the path where kaggle.json is  present in the Google Drive
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/MyDrive/kaggle-keys"

In [ ]:
%%shell

# download dataset from kaggle> URL: https://www.kaggle.com/competitions/AI4Code
kaggle competitions download -c AI4Code

unzip -qq AI4Code.zip
rm -rf AI4Code.zip

 97% 694M/714M [00:05<00:00, 171MB/s]
100% 714M/714M [00:05<00:00, 141MB/s]


##Data source

**Google and X, the moonshot factory, have supplied the dataset, which contains about 160 000 Jupyter notebooks. This is part of a <a href="https://www.kaggle.com/competitions/AI4Code">Kaggle competition</a> to train a model that can rank markdown cells depending on the order of its code cells. 

Code cells are written in python and markdown cells are written in markdown, which is the text formatting langague used in Jupyter.**

In [ ]:
def read_notebook(path):
  return (
      pd.read_json(path, dtype={"cell_type": "category", "source": "str"})
        .assign(id=path.stem)
        .rename_axis("cell_id")
  )

In [ ]:
# Subset of training due to its large size
NUM_TRAIN = 20000

paths_train = list(("train").glob("*.json"))[:NUM_TRAIN]

notebooks_train = [read_notebook(path) for path in tqdm(paths_train, desc="Train NBs")]
# Get notebooks
df_notebooks = (pd.concat())